In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Zero-DCE GitHub 리포지토리 복사
!git clone https://github.com/Li-Chongyi/Zero-DCE.git

Cloning into 'Zero-DCE'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 236 (delta 43), reused 225 (delta 41), pack-reused 0 (from 0)
Receiving objects: 100% (236/236), 30.89 MiB | 24.86 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [3]:
# 작업 디렉토리를 변경
%cd /content/Zero-DCE/Zero-DCE_code

from model import enhance_net_nopool  # Zero-DCE 모델 임포트
import torch

# Zero-DCE 모델 초기화 및 가중치 로드
DCE_net = enhance_net_nopool().cuda()
DCE_net.load_state_dict(torch.load('/content/Zero-DCE/Zero-DCE_code/snapshots/Epoch99.pth'))
DCE_net.eval()

print("Model loaded successfully!")

/content/Zero-DCE/Zero-DCE_code
Model loaded successfully!


<ipython-input-3-55a5aafcb70a>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  DCE_net.load_state_dict(torch.load('/content/Zero-DCE/Zero-DCE_code/snapshots/Epoch99.pth'))


In [4]:
%cd /content

/content


In [5]:
import os
import torch
import torchvision
from PIL import Image
import numpy as np
from model import enhance_net_nopool

# Zero-DCE 모델 초기화 함수
def initialize_model():
    DCE_net = enhance_net_nopool().cuda()
    DCE_net.load_state_dict(torch.load('/content/Zero-DCE/Zero-DCE_code/snapshots/Epoch99.pth'))
    DCE_net.eval()
    return DCE_net

DCE_net = initialize_model()

# Zero-DCE를 사용하여 이미지 개선 함수
def enhance_image(image_path, model, output_folder):
    # 이미지 로드 및 전처리
    data_lowlight = Image.open(image_path).convert("RGB")
    data_lowlight = np.asarray(data_lowlight) / 255.0
    data_lowlight = torch.from_numpy(data_lowlight).float()
    data_lowlight = data_lowlight.permute(2, 0, 1).unsqueeze(0).cuda()

    # Zero-DCE로 개선
    with torch.no_grad():
        _, enhanced_image, _ = model(data_lowlight)

    # 결과 저장
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    torchvision.utils.save_image(enhanced_image, output_path)
    print(f"Enhanced image saved to: {output_path}")

# 폴더의 모든 이미지를 Zero-DCE로 처리
def process_train_images(input_folder, output_folder, model):
    os.makedirs(output_folder, exist_ok=True)
    for image_name in os.listdir(input_folder):
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, image_name)
            enhance_image(image_path, model, output_folder)

# Zero-DCE 모델 초기화
DCE_net = initialize_model()

# YOLO 데이터셋에 Zero-DCE 적용
# train_M
train_input_folder = '/content/drive/MyDrive/Low_Brightness_Moderate/train_M'
train_output_folder = '/content/drive/MyDrive/Low_Brightness_Moderate/train_M'

process_train_images(train_input_folder, train_output_folder, DCE_net)

<ipython-input-5-c2c278a9f2bb>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  DCE_net.load_state_dict(torch.load('/content/Zero-DCE/Zero-DCE_code/snapshots/Epoch99.pth'))

Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/train_M/CCTV_129_png.rf.9737c47850b35adcc26a2d62589ec94d.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/train_M/CCTV_120_png.rf.03eaa4561001fcc3cd99d826facb9776.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/train_M/CCTV_110_png.rf.c861a2a763c1e443e248b25ecd3e6e87.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/train_M/CCTV_108_png.rf.536f9b0a27219da342fda5078abc3738.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/train_M/CCTV_121_png.rf.571ed82e19b7eb2eef03f24a8bba8abc.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/train_M/CCTV_105_png.rf.41735197d28fd9b45545728a01c8f1cb.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/train_M/CCTV_122_png.rf.e04a71cd568b84a6ba84deeefd529536.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Modera

In [6]:
# train_S
train_input_folder = '/content/drive/MyDrive/Low_Brihtness_Severe/train_S'
train_output_folder = '/content/drive/MyDrive/Low_Brihtness_Severe/train_S_Zero'

process_train_images(train_input_folder, train_output_folder, DCE_net)

Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/train_S_Zero/CCTV_11_png.rf.f349e519784d10cb50598f2f24658cc6.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/train_S_Zero/CCTV_105_png.rf.41735197d28fd9b45545728a01c8f1cb.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/train_S_Zero/CCTV_119_png.rf.e89104200cfd914303e66728ebe899c7.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/train_S_Zero/CCTV_117_png.rf.7b8110f8ef4edd8668edcab545c3741e.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/train_S_Zero/CCTV_108_png.rf.536f9b0a27219da342fda5078abc3738.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/train_S_Zero/CCTV_118_png.rf.adeeefefde403b577f105e4459d40a20.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/train_S_Zero/CCTV_103_png.rf.320bba6f67e892e09f373e9fc3da5071.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brih

In [7]:
# train_E
train_input_folder = '/content/drive/MyDrive/Low_Brightness_Extreme/train_E'
train_output_folder = '/content/drive/MyDrive/Low_Brightness_Extreme/train_E_Zero'

process_train_images(train_input_folder, train_output_folder, DCE_net)

Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/train_E_Zero/CCTV_108_png.rf.536f9b0a27219da342fda5078abc3738.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/train_E_Zero/CCTV_156_png.rf.bbe7598d7433c4e7d82a8e0229670252.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/train_E_Zero/CCTV_149_png.rf.5ef01304a5284293be80872c710e3baf.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/train_E_Zero/CCTV_14_png.rf.a247733b2f0319038bb24411ff8c0460.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/train_E_Zero/CCTV_139_png.rf.235861803d24c96eca6d8d6731eb4d9f.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/train_E_Zero/CCTV_204_png.rf.733b99d192cf268723a9d30f0a76599b.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/train_E_Zero/CCTV_184_png.rf.cfac2acc423269904470331fcb0be789.jpg
Enhanced image saved to: /content/drive/My

In [8]:
# valid_M
train_input_folder = '/content/drive/MyDrive/Low_Brightness_Moderate/valid_M'
train_output_folder = '/content/drive/MyDrive/Low_Brightness_Moderate/valid_M_Zero'

process_train_images(train_input_folder, train_output_folder, DCE_net)

Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/valid_M_Zero/CCTV_101_png.rf.dcf9369b02ef926bc7697b3bf36171c4.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/valid_M_Zero/CCTV_128_png.rf.2f3ceddb3e1d4fc1c204256b00835fc5.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/valid_M_Zero/CCTV_10_png.rf.ec25da7011373f765146a486de805363.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/valid_M_Zero/CCTV_114_png.rf.1e2e4e22be216082d18f3f13ac67c18a.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/valid_M_Zero/CCTV_111_png.rf.50088322dbea4274d8b77b6dd09f36d5.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/valid_M_Zero/CCTV_136_png.rf.27db5af36c6157e9e8c8947ee2f4ae44.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Moderate/valid_M_Zero/CCTV_112_png.rf.204ae122b632fc35d69df0ba6601cb52.jpg
Enhanced image saved to: /content/d

In [9]:
# valid_S
train_input_folder = '/content/drive/MyDrive/Low_Brihtness_Severe/valid_S'
train_output_folder = '/content/drive/MyDrive/Low_Brihtness_Severe/valid_S_Zero'

process_train_images(train_input_folder, train_output_folder, DCE_net)

Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/valid_S_Zero/CCTV_250_png.rf.4c8216ff8bdfee9ed6907b0b41818577.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/valid_S_Zero/CCTV_12_png.rf.94d1944f3f7fabee67fe2c3b74164499.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/valid_S_Zero/CCTV_412_png.rf.45419d6a95817ddc200dc2a5f670357d.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/valid_S_Zero/CCTV_394_png.rf.9069bc7e904c09ee2631546a67e9bddc.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/valid_S_Zero/CCTV_371_png.rf.3fbe785330bf1ee778758042a795c037.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/valid_S_Zero/CCTV_353_png.rf.6b1be1c9d21db338ebca021536d2358c.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brihtness_Severe/valid_S_Zero/CCTV_300_png.rf.159fdff6b4ce67a0b7f18805a773c01e.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brih

In [13]:
# valid_E
train_input_folder = '/content/drive/MyDrive/Low_Brightness_Extreme/valid_E'
train_output_folder = '/content/drive/MyDrive/Low_Brightness_Extreme/valid_E_Zero'

process_train_images(train_input_folder, train_output_folder, DCE_net)

Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/valid_E_Zero/CCTV_101_png.rf.dcf9369b02ef926bc7697b3bf36171c4.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/valid_E_Zero/CCTV_130_png.rf.eee2e7483e27cb20755a0d7839d84fbd.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/valid_E_Zero/CCTV_173_png.rf.229b827773d12f4fb67f41a00a1d76c0.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/valid_E_Zero/CCTV_114_png.rf.1e2e4e22be216082d18f3f13ac67c18a.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/valid_E_Zero/CCTV_112_png.rf.204ae122b632fc35d69df0ba6601cb52.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/valid_E_Zero/CCTV_151_png.rf.ef24484ece265bde608b4d8dc1457ba2.jpg
Enhanced image saved to: /content/drive/MyDrive/Low_Brightness_Extreme/valid_E_Zero/CCTV_12_png.rf.94d1944f3f7fabee67fe2c3b74164499.jpg
Enhanced image saved to: /content/drive/My